### Install/Import Dependencies
Here we install or import the libraries needed. Adjust to match your environment.
Make sure that `langchain-core`, `langgraph`, and `langchain_openai` are installed (or locally available).

In [4]:
import mlflow
from typing import Literal
from langchain_core.messages import AIMessage, ToolCall
from langchain_core.outputs import ChatGeneration, ChatResult
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from dotenv import load_dotenv
load_dotenv()

mlflow.langchain.autolog()

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("LangGraph")


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1742932410518, experiment_id='1', last_update_time=1742932410518, lifecycle_stage='active', name='LangGraph', tags={}>

### Define our Tool & Graph
Below is the code snippet provided in your request. We define a simple tool to get weather (with limited city options) and create a ReAct-style agent using LangGraph.

In [5]:
@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"

# Instantiate the LLM
llm = ChatOpenAI(model="gpt-4o-mini")  # placeholder model name

# Create the ReAct agent
tools = [get_weather]
graph = create_react_agent(llm, tools)


### Invoke the Graph
We now call `graph.invoke` with a user request about the weather in SF. 

In [6]:
result = graph.invoke({
    "messages": [
        {"role": "user", "content": "what is the weather in sf?"}
    ]
})
print("Agent response:", result)

Agent response: {'messages': [HumanMessage(content='what is the weather in sf?', additional_kwargs={}, response_metadata={}, id='34f8ccb6-0314-4485-9532-e13e880e82e8'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_YAQaz2rUwYIIVUMThKJYEpEu', 'function': {'arguments': '{"city":"sf"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 58, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_27322b4e16', 'id': 'chatcmpl-BF4drrTQpp0Q7HuxUhNcsrReL6WKx', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-bb0becc1-6426-492a-9982-edcb0c1aa27b-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'call_YAQaz2rUwYIIVUMThKJYEpE

Trace(request_id=8a5aea03de7a49038b3bcc940728ad3f)

In [ ]:
https://www.mlflow.org/docs/latest/tracing/api/manual-instrumentation/